In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score,confusion_matrix

In [2]:
df = pd.read_csv('./Dataset/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 889 entries, 0 to 888
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  889 non-null    int64  
 1   Survived     889 non-null    int64  
 2   Pclass       889 non-null    int64  
 3   Name         889 non-null    object 
 4   Sex          889 non-null    object 
 5   Age          889 non-null    float64
 6   SibSp        889 non-null    int64  
 7   Parch        889 non-null    int64  
 8   Ticket       889 non-null    object 
 9   Fare         889 non-null    float64
 10  Cabin        202 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.5+ KB


# Encoding of categorical variables

In [4]:
enc = LabelEncoder()
df['Sex'] = enc.fit_transform(df.Sex)
df['Embarked'] = enc.fit_transform(df.Embarked)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 889 entries, 0 to 888
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  889 non-null    int64  
 1   Survived     889 non-null    int64  
 2   Pclass       889 non-null    int64  
 3   Name         889 non-null    object 
 4   Sex          889 non-null    int32  
 5   Age          889 non-null    float64
 6   SibSp        889 non-null    int64  
 7   Parch        889 non-null    int64  
 8   Ticket       889 non-null    object 
 9   Fare         889 non-null    float64
 10  Cabin        202 non-null    object 
 11  Embarked     889 non-null    int32  
dtypes: float64(2), int32(2), int64(5), object(3)
memory usage: 76.5+ KB


In [5]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

# Data Cleaning

In [6]:
df = df.drop(columns = ['Cabin', 'PassengerId', 'Name', 'Ticket'])
df.columns

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked'],
      dtype='object')

# Model Function

In [7]:
clf=BernoulliNB()

def model(x, y, rstate):
    X_train, X_test, y_train, y_test=train_test_split(x, y,test_size=0.3, random_state=rstate)
    y_pred = clf.fit(X_train, y_train).predict(X_test)
    acc = accuracy_score(y_test, y_pred, normalize=True)
    confm = confusion_matrix(y_test, y_pred)
    return [acc, confm]

# Splitting the data to feed into the model

In [8]:
rstate=-1
models = []

for enum, i in enumerate(df.columns[1:]):
    if i == 'Age' or i == 'Fare':
        continue
    y = df.iloc[:, enum+1:enum+2]
    x = df.drop(columns = y.columns)
    rstate += 1
    models.append([model(x, y, rstate), y.columns[0]])

c:\python3764\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
c:\python3764\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
c:\python3764\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
c:\python3764\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
c:\python3764\lib\site-packages\skle

# Model Outcome

In [9]:
for _ in models:
    print(f'{_[1]} vs All\n')
    print(f'Accuracy: {_[0][0]}')
    print(f'Confusion Matrix: \n {_[0][1]}\n')
    print(f"Out of total 267 records only {np.trace(_[0][1])} are successfully classified, giving a model accuracy of {int(_[0][0]*100)}%.")
    print('\n-----------------------\n')

Pclass vs All

Accuracy: 0.5917602996254682
Confusion Matrix: 
 [[ 30   8  32]
 [  9  10  30]
 [ 24   6 118]]

Out of total 267 records only 158 are successfully classified, giving a model accuracy of 59%.

-----------------------

Sex vs All

Accuracy: 0.7602996254681648
Confusion Matrix: 
 [[ 48  49]
 [ 15 155]]

Out of total 267 records only 203 are successfully classified, giving a model accuracy of 76%.

-----------------------

SibSp vs All

Accuracy: 0.6891385767790262
Confusion Matrix: 
 [[167  25   0   0   0   0   0]
 [ 31  17   0   0   0   0   0]
 [  5   7   0   0   0   0   0]
 [  4   3   0   0   0   0   0]
 [  2   2   0   0   0   0   0]
 [  0   1   0   0   0   0   0]
 [  1   2   0   0   0   0   0]]

Out of total 267 records only 184 are successfully classified, giving a model accuracy of 68%.

-----------------------

Parch vs All

Accuracy: 0.7191011235955056
Confusion Matrix: 
 [[183  16   0   0   0]
 [ 34   9   0   0   0]
 [ 18   3   0   0   0]
 [  2   0   0   0   0]
 [  